In [3]:
import os
os.makedirs("C:/Users/Mao-k/網頁爬蟲/Final_Project_Web_Crawler_2025_Spring/site", exist_ok=True)


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re
import csv

# 儲存資料夾路徑
base_path = r"C:\Users\Mao-k\網頁爬蟲\Final_Project_Web_Crawler_2025_Spring"
save_path = r"C:\Users\Mao-k\網頁爬蟲\Final_Project_Web_Crawler_2025_Spring\Web_crawel_Data\LetterBox_movie"
data_folder = r"C:\Users\Mao-k\網頁爬蟲\Final_Project_Web_Crawler_2025_Spring\data"
site = os.path.join(base_path, "Crawler_code", "site")
os.makedirs(site, exist_ok=True)
os.makedirs(save_path, exist_ok=True)
os.makedirs(data_folder, exist_ok=True)

# 設定 Selenium
options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1200,900")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

search_query = "玩命關頭4"
safe_query = search_query.strip().replace(" ", "_")

try:
    # Step 1：搜尋電影
    search_url = f"https://letterboxd.com/search/{search_query}/"
    driver.get(search_url)
    time.sleep(3)

    # Step 2：點擊第一筆結果
    movie_link = driver.find_element(By.XPATH, '//ul[@class="results"]/li//a[contains(@href, "/film/")]')
    movie_link.click()
    time.sleep(3)

    # Step 3：解析頁面
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Step 4：擷取基本資料
    title = soup.find("h1", class_="headline-1").text.strip() if soup.find("h1", class_="headline-1") else "N/A"

    year_tag = soup.select_one("div.metablock div.releaseyear a")
    year = year_tag.text.strip() if year_tag else "N/A"

    description_tag = soup.find("meta", attrs={"name": "description"})
    description = description_tag["content"] if description_tag else "N/A"

    poster_a_tag = soup.find("a", attrs={"data-js-trigger": "postermodal"})
    poster_url = poster_a_tag["href"] if poster_a_tag else "N/A"

    # Step 5：平均分數與總評價數
    average_score = "N/A"
    total_ratings = "N/A"
    rating_block = soup.select_one('section.ratings-histogram-chart')
    if rating_block:
        avg_tag = rating_block.select_one('span.average-rating a')
        if avg_tag:
            title_text = avg_tag.get('data-original-title', '')
            match = re.search(r'([\d.]+).*?based on ([\d,]+)', title_text)
            if match:
                average_score = match.group(1)
                total_ratings = match.group(2).replace(',', '')

    # Step 6：擷取各星等的評價數量
    star_ratings = {
        '0.5': 0, '1': 0, '1.5': 0, '2': 0, '2.5': 0,
        '3': 0, '3.5': 0, '4': 0, '4.5': 0, '5': 0
    }

    rating_map = {
        'half-★': '0.5', '★': '1', '★½': '1.5', '★★': '2', '★★½': '2.5',
        '★★★': '3', '★★★½': '3.5', '★★★★': '4', '★★★★½': '4.5', '★★★★★': '5'
    }

    for li in soup.select('li.rating-histogram-bar'):
        a_tag = li.find('a')
        if a_tag:
            tooltip = a_tag.get("data-original-title", "")
            match = re.search(r'([\d,]+)\s+([\S]+) ratings', tooltip)
            if match:
                count = int(match.group(1).replace(",", ""))
                stars_text = match.group(2)
                rating_value = rating_map.get(stars_text)
                if rating_value:
                    star_ratings[rating_value] = count

    # Step 7：建立 DataFrame 並匯出 CSV
    movie_data = [{
        "電影標題": title,
        "上映年份": year,
        "電影描述": description,
        "海報圖片": poster_url,
        "平均評分": average_score,
        "評分總數": total_ratings,
        "0.5星": star_ratings['0.5'],
        "1星": star_ratings['1'],
        "1.5星": star_ratings['1.5'],
        "2星": star_ratings['2'],
        "2.5星": star_ratings['2.5'],
        "3星": star_ratings['3'],
        "3.5星": star_ratings['3.5'],
        "4星": star_ratings['4'],
        "4.5星": star_ratings['4.5'],
        "5星": star_ratings['5']
    }]

    df = pd.DataFrame(movie_data)
    csv_path = os.path.join(save_path, f"{safe_query}_movie_info.csv")
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"✅ 成功儲存：{csv_path}")

    # Step 8：擷取評論資料
    reviews_url = driver.current_url.rstrip("/") + "/reviews/by/activity/"
    print(f"📄 前往評論頁面：{reviews_url}")
    driver.get(reviews_url)
    wait = WebDriverWait(driver, 20)

    print("⌛ 等待評論清單載入...")
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "viewing-list")))

    review_list = driver.find_elements(By.CSS_SELECTOR, ".viewing-list .listitem article")
    print(f"📝 找到 {len(review_list)} 筆評論")

    filename = f"{safe_query}_reviews.csv"
    output_path = os.path.join(save_path, filename)

    def convert_rating_to_number(rating_str):
        if not rating_str:
            return ""
        stars = rating_str.count("★")
        half = "½" in rating_str
        return stars + 0.5 if half else stars

    with open(output_path, mode="w", encoding="utf-8", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Reviewer", "Rating", "Date", "Content", "Likes"])

        for review in review_list:
            try:
                reviewer = review.find_element(By.CSS_SELECTOR, ".displayname").text
            except:
                reviewer = "無名稱"

            try:
                rating_text = review.find_element(By.CSS_SELECTOR, "span.rating").text.strip()
                rating = convert_rating_to_number(rating_text)
            except:
                rating = ""

            try:
                date = review.find_element(By.CSS_SELECTOR, "time.timestamp").get_attribute("datetime")
            except:
                date = ""

            try:
                content = review.find_element(By.CSS_SELECTOR, "div.js-review-body").text.strip()
            except:
                content = ""

            try:
                likes_text = review.find_element(By.CSS_SELECTOR, "span._count_8kxo2_22").text
                likes = int(re.search(r"\d+", likes_text.replace(",", "")).group())
            except:
                likes = 0

            print(f"{reviewer} | {rating} | {date} | 👍 {likes}")
            writer.writerow([reviewer, rating, date, content, likes])

    print(f"✅ 評論資料已儲存至：{output_path}")

finally:
    driver.quit()


def convert_csv_to_json(search_query):
    safe_query = search_query.strip().replace(" ", "_")
    
    movie_csv = os.path.join(save_path, f"{safe_query}_movie_info.csv")
    reviews_csv = os.path.join(save_path, f"{safe_query}_reviews.csv")
    movie_json = os.path.join(site, f"{safe_query}_movie_info.json")
    reviews_json = os.path.join(site, f"{safe_query}_reviews.json")

    os.makedirs(site, exist_ok=True)

    try:
        df_movie = pd.read_csv(movie_csv, encoding='utf-8-sig')
        df_reviews = pd.read_csv(reviews_csv, encoding='utf-8-sig')

        df_movie.to_json(movie_json, orient='records', force_ascii=False)
        df_reviews.to_json(reviews_json, orient='records', force_ascii=False)

        print(f"✅ 已轉換 JSON 並儲存至：\n{movie_json}\n{reviews_json}")
    except FileNotFoundError as e:
        print(f"❌ 找不到檔案：{e.filename}")



✅ 成功儲存：C:\Users\Mao-k\網頁爬蟲\Final_Project_Web_Crawler_2025_Spring\Web_crawel_Data\LetterBox_movie\玩命關頭4_movie_info.csv
📄 前往評論頁面：https://letterboxd.com/film/fast-furious/reviews/by/activity/
⌛ 等待評論清單載入...
📝 找到 12 筆評論
Willow Maclay | 3 | 2015-04-05 | 👍 2125
Josh Larsen | 3.5 | 2015-03-28 | 👍 1933
Josh Lewis | 2 | 2015-03-25 | 👍 1679
Andy Young | 3.5 | 2021-06-19 | 👍 1186
Lucy | 2 | 2021-06-22 | 👍 970
Matt Singer | 3.5 | 2021-06-24 | 👍 838
Bryan Espitia | 3.5 | 2021-05-21 | 👍 651
Bryan Espitia | 3.5 | 2023-05-06 | 👍 642
demi adejuyigbe |  | 2021-06-19 | 👍 499
Wood | 3 | 2015-04-08 | 👍 424
Framesofnick | 2 | 2020-07-15 | 👍 349
Leah 🥀 | 3 | 2022-03-13 | 👍 330
✅ 評論資料已儲存至：C:\Users\Mao-k\網頁爬蟲\Final_Project_Web_Crawler_2025_Spring\Web_crawel_Data\LetterBox_movie\玩命關頭4_reviews.csv
